In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [21]:
data = pd.read_csv('data/train.csv')
data.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data = np.array(data)
m, n = data.shape
print(m, n)
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.


42000 785


In [23]:
#test cell
np.random.rand(10, 784) - 0.5

array([[ 0.3007428 , -0.35466005,  0.18832975, ..., -0.08893447,
        -0.01219421,  0.45553953],
       [-0.06921946,  0.4969576 ,  0.11405489, ..., -0.0511308 ,
         0.3733335 ,  0.35046284],
       [ 0.22700196, -0.13386474, -0.40505469, ...,  0.088808  ,
        -0.46569233,  0.20014214],
       ...,
       [ 0.42187449, -0.09477248, -0.44643129, ...,  0.18391424,
         0.1079843 ,  0.17053795],
       [-0.18096781,  0.37115989,  0.2705171 , ...,  0.36988038,
        -0.35725286,  0.06004289],
       [ 0.37021163, -0.44127392, -0.16283583, ..., -0.2650258 ,
         0.00974448,  0.45736453]])

In [30]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def LReLU(Z):
    return np.where(Z > 0, Z, 0.01 * Z)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = LReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    print(X.shape)
    print(A1.shape)
    print(A2.shape)
    print(A2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * (Z1 > 0)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    print(dZ2.shape)
    print(dW2.shape)
    print(db2.shape)
    print(dZ1.shape)
    print(dW1.shape)
    print(db1.shape)
    print(db2, db1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2



In [31]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        if i % 10 == 0:
            print("Iteration: ", i)
            print(A2)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [32]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.3, 1)

(784, 41000)
(10, 41000)
(10, 41000)
[[0.22915304 0.11799041 0.37627357 ... 0.05441824 0.03244524 0.09270452]
 [0.01684601 0.14623447 0.03784497 ... 0.16706708 0.17172349 0.18215486]
 [0.03100354 0.20889182 0.04586253 ... 0.05666441 0.05737928 0.17942083]
 ...
 [0.05253957 0.01582738 0.18751368 ... 0.08912959 0.02485177 0.04376778]
 [0.02544682 0.01975695 0.01196127 ... 0.00259586 0.01767748 0.0139384 ]
 [0.61723654 0.22207507 0.20679166 ... 0.03482    0.29252553 0.03803144]]
(10, 41000)
(10, 10)
()
(10, 41000)
(10, 784)
()
(784, 41000)
(10, 41000)
(10, 41000)
[[0.31459541 0.08959262 0.37525533 ... 0.03826122 0.02798405 0.06172346]
 [0.03628118 0.15047924 0.044373   ... 0.16553397 0.15746965 0.13894207]
 [0.09196606 0.2407881  0.10119596 ... 0.10196409 0.06281547 0.21769128]
 ...
 [0.06866403 0.01854926 0.16068046 ... 0.092209   0.03627759 0.08596517]
 [0.11071872 0.04648388 0.05741905 ... 0.01264196 0.04018567 0.04669274]
 [0.31358453 0.14676688 0.09909689 ... 0.02301077 0.26500374 0.